In [1]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,floatX=float32'

In [2]:
import theano
import theano.tensor as T
import theano.tensor.nnet as NN
import numpy as NP
import numpy.random as RNG
import h5py
import dataUtils
from collections import OrderedDict
from Wrapper import Model, rmsprop
%matplotlib inline
import matplotlib.pyplot as plt

Using gpu device 0: GeForce GTX 850M (CNMeM is disabled)


In [3]:
from dataUtils import BouncingMNIST
seq_len = 5
batch_size = 32
image_size = 100
bmnist = BouncingMNIST(batch_size, image_size, 'train/inputs', 'train/targets', clutter_size_max = 14, scale_range = 0.1)

In [49]:
def get_conv_out_dim(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    output_dim = ((img_row - filter_row) / stride_row + 1) * ((img_col - filter_col) / stride_col + 1) * nr_filters * 1
    return output_dim

def get_conv_out_shape(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    return (((img_row - filter_row) / stride_row + 1), ((img_col - filter_col) / stride_col + 1))

def _step(cur_in):
    
    conv1_shape = (32, 1, 5, 5, 2, 2)
    conv1_h = model.conv(cur_in = cur_in, name = 'conv1', shape=conv1_shape)
    conv1_out_shape = get_conv_out_shape(img_shape, conv1_shape)
    conv2_shape = (32, 32, 5, 5, 2, 2)
    conv2_out_dim = get_conv_out_dim(conv1_out_shape, conv2_shape)
    conv2_h = model.conv(cur_in = conv1_h, name = 'conv2', shape=conv2_shape)
    fc1 = model.fc(cur_in = conv2_h.flatten(2), name = 'fc1', shape=(conv2_out_dim, 100))
    fc2 = model.fc(cur_in = fc1, name = 'fc2', shape=(100, out_dim))
    return fc2
    

img_shape = (100, 100)
out_dim = 10
model = Model()
img_in = T.tensor4()
seg_img = T.tensor4()
img_tar = T.matrix()
img_out = _step(img_in)
_EPSI = 1e-6
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar), axis=1))
cost_cls = cost 
cost_sal = [None]*10
for i in xrange(10):
    sal_img = T.grad(T.sum(img_out[:,i]), img_in)
    sal_img = sal_img / (T.sum(sal_img)+_EPSI)
    cost_sal[i] = T.mean((sal_img-seg_img[:,i:i+1])**2)
    cost += cost_sal[i]
    
#test_func = theano.function([img_in, img_tar, seg_img], cost, allow_input_downcast=True)
train_func = theano.function([img_in, img_tar, seg_img], [cost_cls, cost, img_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
#predict_func = theano.function([img_in], img_out, allow_input_downcast=True)

ValueError: ('You cannot drop a non-broadcastable dimension.', ((False, False, False, False, False), (0, 4, 1, 2)))

In [30]:
tt = test_func(data, label, seg)

In [48]:
T.grad(cost, model.weightsPack.getW_list())

ValueError: ('You cannot drop a non-broadcastable dimension.', ((False, False, False, False, False), (0, 4, 1, 2)))

In [36]:
model.layersPack.keys()

['conv2', 'conv1', 'fc2', 'fc1']

In [11]:
data, label, seg=bmnist.GetStaticBatch(num_digits=RNG.randint(5))

/usr/lib64/python2.7/site-packages/scipy/ndimage/interpolation.py:549: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [19]:
conv1_shape = (32, 1, 5, 5, 2, 2)
conv1_out_shape = get_conv_out_shape(img_shape, conv1_shape)
conv2_shape = (32, 32, 5, 5, 2, 2)
conv2_out_dim = get_conv_out_dim(conv1_out_shape, conv2_shape)
print conv2_out_dim/32

15488


In [ ]:
def get_conv_out_dim(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    output_dim = ((img_row - filter_row) / stride_row + 1) * ((img_col - filter_col) / stride_col + 1) * nr_filters * nr_channels
    return output_dim

def get_conv_out_shape(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    return (((img_row - filter_row) / stride_row + 1), ((img_col - filter_col) / stride_col + 1))

In [ ]:
from ipy_progressbar import ProgressBar
for i in xrange(10):
    #pb = ProgressBar(500, title='Epoch '+str(i), key='outer')
    #for j in pb:
    for j in xrange(500):
        data, label, seg=bmnist.GetStaticBatch(num_digits=RNG.randint(5))
        n_cost_cls, n_cost, _ = train_func(data, label, seg) 
        print 'Epoch = ', str(i), ' Batch = ', str(j), ' Cost CLS = ', n_cost_cls, ' Cost SUM = ', n_cost
        #pb_inner = ProgressBar(1, title='Inner', key='inner')
    print 'Epoch = ', str(i), ' Cost = ', n_cost
    

In [ ]:
NP.shape(seg)

In [ ]:
help(print)

In [ ]:
label[4]

In [ ]:
np.mean(seg[4][6])

In [ ]:
plt.imshow(data[4][0])
plt.show()
plt.imshow(seg[4][7])
plt.show()

In [ ]:
''' 
load data 
'''
in_dim = 49
out_dim = 49
batch_size = 16

X_train, Y_train=dataUtils.read_txt('ptb.train.txt')
#X_test, Y_test=dataUtils.read_txt('ptb.test.txt')


In [ ]:
def _step(trash, cur_in, rec_in1, rec_in2, rec_in3):
    
    gru1_h = model.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 20))
    gru2_h = model.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(20, 20))
    gru3_h = model.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(20, 20))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return [fc1] + [gru1_h, gru2_h, gru3_h]
    
model = Model()
char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20))])
char_out = sc[0].dimshuffle(1, 0, 2)
_EPSI = 1e-6
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(char_out, _EPSI, 1.0 - _EPSI), char_tar), axis=2))

#cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))
train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
predict_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)

In [ ]:
def _step(cur_in, trash, rec_in1, rec_in2, rec_in3):
    
    gru1_h = model2.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 100))
    gru2_h = model2.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(100, 100))
    gru3_h = model2.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(100, 100))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return [fc1] + [gru1_h, gru2_h, gru3_h]
    
model2 = Model()


char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20))])
char_out = sc[0].dimshuffle(1, 0, 2)

#cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))

#train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
#Jpredict_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)
jacobi = T.jacobian(char_out[0, :50, 0:3].flatten(), char_in)
jacobi_func = theano.function([char_in, starts], jacobi, allow_input_downcast=True)

In [ ]:
model2.weightsPack = model.weightsPack
model2.layersPack = model.layersPack

In [ ]:
str(0.5555)

In [ ]:
pb = ProgressBar(40000, title='Epoch '+str(1), key='outer')
for i in pb:
    pb_inner = ProgressBar(1, title='Inner', key='inner')

In [ ]:
len(X_train)

In [ ]:

batch_size = 64
pb = ProgressBar(len(X_train), title='Epoch '+str(i), key='outer')
for i in xrange(10):
    print 'epoch ', i

    for j in xrange(len(X_train)/batch_size):
        train_cost, _ = train_func(X_train[j*batch_size:(j+1)*batch_size], X_train[j*batch_size:(j+1)*batch_size,0], Y_train[j*batch_size:(j+1)*batch_size]) 
        pb_inner = ProgressBar(1, title='Inner', key='inner')

In [ ]:
J_predict = predict_func(X_train[0:1], X_train[0:1, 0])
J_in = NP.concatenate((X_train[0:1, 0:1, :], J_predict[0:1, :-1, :]), axis=1)
v_jacobi = jacobi_func(X_train[0:1], X_train[0:1, 0])

In [ ]:
print J_predict[0, 3]
print Y_train[0, 3]

In [ ]:
print J_predict[0, 3]
print Y_train[0, 3]

In [ ]:
p_jacobi = jacobi_func(X_train[0:1], X_train[0:1, 0])
#print p_jacobi

In [ ]:
NP.shape(v_jacobi)

In [ ]:
NP.sum(abs(v_jacobi[30]), axis=2)

In [ ]:
help(ProgressBar)

In [ ]:
pb = ProgressBar(1000, title='Outer', key='outer')
for i in pb:
    pb_inner = ProgressBar(1, title='Inner', key='inner')
    print 'hhh'